In [1]:
import numpy as np
from skimage.transform import resize
from skimage.io import imsave
from skimage.color import rgb2lab,lab2rgb,gray2rgb
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D , UpSampling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import imagenet_utils
from sklearn.metrics import confusion_matrix
import itertools
import cv2
import os
import shutil
import random
import glob
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [38]:
mobile = tf.keras.applications.mobilenet.MobileNet()

In [39]:
mobile.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, 112, 112, 32

In [40]:
x = mobile.layers[-16].output
output = (x)

In [30]:
newmodel = Model(inputs=mobile.input, outputs=output)

In [31]:
newmodel.summary()


Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, 112, 112, 32)      0   

In [21]:
for i, layer in enumerate(newmodel.layers):

        layer.trainable = False

In [22]:
newmodel.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, 112, 112, 32)      0   

In [23]:
path = 'downloads/'

In [52]:
train_datagen = ImageDataGenerator(rescale=1. / 255)

In [53]:
train = train_datagen.flow_from_directory(path, target_size=(224, 224), batch_size=32, class_mode=None)

Found 563 images belonging to 1 classes.


In [54]:
X =[]
Y =[]
for img in train[0]:
    try:
        
        lab = rgb2lab(img)
        X.append(lab[:,:,0]) 
        Y.append(lab[:,:,1:] / 128) #A and B values range from -127 to 128, 
      #so we divide the values by 128 to restrict values to between -1 and 1.
    except:
        print('error')

In [55]:
X = np.array(X)
Y = np.array(Y)
X = X.reshape(X.shape+(1,)) #dimensions to be the same for X and Y
print(X.shape)
print(Y.shape)

(32, 224, 224, 1)
(32, 224, 224, 2)


In [56]:
vggfeatures = []
for i, sample in enumerate(X):
    sample = gray2rgb(sample)
    sample = sample.reshape((1,224,224,3))
    prediction = newmodel.predict(sample)
    prediction = prediction.reshape((7,7,512))
    vggfeatures.append(prediction)
vggfeatures = np.array(vggfeatures)
print(vggfeatures.shape)

(32, 7, 7, 512)


In [57]:
model = Sequential()

model.add(Conv2D(256, (3,3), activation='relu', padding='same', input_shape=(7,7,512)))
model.add(Conv2D(128, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(16, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))
model.add(UpSampling2D((2, 2)))
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 7, 7, 256)         1179904   
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 7, 7, 128)         295040    
_________________________________________________________________
up_sampling2d_10 (UpSampling (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 14, 14, 64)        73792     
_________________________________________________________________
up_sampling2d_11 (UpSampling (None, 28, 28, 64)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 28, 28, 32)        18464     
_________________________________________________________________
up_sampling2d_12 (UpSampling (None, 56, 56, 32)       

In [58]:
model.compile(optimizer='Adam', loss='mse' , metrics=['accuracy'])
model.fit(vggfeatures, Y, verbose=1, epochs=500, batch_size=128)

Epoch 1/500
1/1 [==============================] - 1s 681ms/step - loss: 0.3449 - accuracy: 0.4735
Epoch 2/500
1/1 [==============================] - 0s 52ms/step - loss: 0.7636 - accuracy: 0.5247
Epoch 3/500
1/1 [==============================] - 0s 52ms/step - loss: 0.8002 - accuracy: 0.4767
Epoch 4/500
1/1 [==============================] - 0s 52ms/step - loss: 0.7133 - accuracy: 0.4760
Epoch 5/500
1/1 [==============================] - 0s 49ms/step - loss: 0.5201 - accuracy: 0.4754
Epoch 6/500
1/1 [==============================] - 0s 51ms/step - loss: 0.4328 - accuracy: 0.4774
Epoch 7/500
1/1 [==============================] - 0s 51ms/step - loss: 0.2331 - accuracy: 0.4782
Epoch 8/500
1/1 [==============================] - 0s 49ms/step - loss: 0.0643 - accuracy: 0.4808
Epoch 9/500
1/1 [==============================] - 0s 51ms/step - loss: 0.0307 - accuracy: 0.5035
Epoch 10/500
1/1 [==============================] - 0s 49ms/step - loss: 0.0250 - accuracy: 0.5080
Epoch 11/500
1/1 [

1/1 [==============================] - 0s 47ms/step - loss: 0.0056 - accuracy: 0.7127
Epoch 166/500
1/1 [==============================] - 0s 47ms/step - loss: 0.0056 - accuracy: 0.7187
Epoch 167/500
1/1 [==============================] - 0s 46ms/step - loss: 0.0056 - accuracy: 0.7100
Epoch 168/500
1/1 [==============================] - 0s 49ms/step - loss: 0.0056 - accuracy: 0.7196
Epoch 169/500
1/1 [==============================] - 0s 47ms/step - loss: 0.0056 - accuracy: 0.7089
Epoch 170/500
1/1 [==============================] - 0s 46ms/step - loss: 0.0056 - accuracy: 0.7200
Epoch 171/500
1/1 [==============================] - 0s 47ms/step - loss: 0.0055 - accuracy: 0.7126
Epoch 172/500
1/1 [==============================] - 0s 46ms/step - loss: 0.0055 - accuracy: 0.7193
Epoch 173/500
1/1 [==============================] - 0s 46ms/step - loss: 0.0054 - accuracy: 0.7167
Epoch 174/500
1/1 [==============================] - 0s 48ms/step - loss: 0.0054 - accuracy: 0.7171
Epoch 175/500


1/1 [==============================] - 0s 47ms/step - loss: 0.0044 - accuracy: 0.7338
Epoch 248/500
1/1 [==============================] - 0s 47ms/step - loss: 0.0044 - accuracy: 0.7164
Epoch 249/500
1/1 [==============================] - 0s 47ms/step - loss: 0.0042 - accuracy: 0.7317
Epoch 250/500
1/1 [==============================] - 0s 47ms/step - loss: 0.0042 - accuracy: 0.7317
Epoch 251/500
1/1 [==============================] - 0s 46ms/step - loss: 0.0043 - accuracy: 0.7220
Epoch 252/500
1/1 [==============================] - 0s 49ms/step - loss: 0.0044 - accuracy: 0.7340
Epoch 253/500
1/1 [==============================] - 0s 50ms/step - loss: 0.0043 - accuracy: 0.7198
Epoch 254/500
1/1 [==============================] - 0s 49ms/step - loss: 0.0042 - accuracy: 0.7301
Epoch 255/500
1/1 [==============================] - 0s 48ms/step - loss: 0.0042 - accuracy: 0.7331
Epoch 256/500
1/1 [==============================] - 0s 49ms/step - loss: 0.0044 - accuracy: 0.7198
Epoch 257/500


1/1 [==============================] - 0s 47ms/step - loss: 0.0036 - accuracy: 0.7460
Epoch 330/500
1/1 [==============================] - 0s 46ms/step - loss: 0.0036 - accuracy: 0.7429
Epoch 331/500
1/1 [==============================] - 0s 47ms/step - loss: 0.0036 - accuracy: 0.7471
Epoch 332/500
1/1 [==============================] - 0s 47ms/step - loss: 0.0036 - accuracy: 0.7411
Epoch 333/500
1/1 [==============================] - 0s 47ms/step - loss: 0.0036 - accuracy: 0.7483
Epoch 334/500
1/1 [==============================] - 0s 49ms/step - loss: 0.0037 - accuracy: 0.7388
Epoch 335/500
1/1 [==============================] - 0s 51ms/step - loss: 0.0037 - accuracy: 0.7493
Epoch 336/500
1/1 [==============================] - 0s 48ms/step - loss: 0.0037 - accuracy: 0.7376
Epoch 337/500
1/1 [==============================] - 0s 47ms/step - loss: 0.0037 - accuracy: 0.7497
Epoch 338/500
1/1 [==============================] - 0s 46ms/step - loss: 0.0036 - accuracy: 0.7432
Epoch 339/500


1/1 [==============================] - 0s 47ms/step - loss: 0.0032 - accuracy: 0.7638
Epoch 412/500
1/1 [==============================] - 0s 47ms/step - loss: 0.0032 - accuracy: 0.7683
Epoch 413/500
1/1 [==============================] - 0s 47ms/step - loss: 0.0032 - accuracy: 0.7632
Epoch 414/500
1/1 [==============================] - 0s 46ms/step - loss: 0.0031 - accuracy: 0.7673
Epoch 415/500
1/1 [==============================] - 0s 47ms/step - loss: 0.0031 - accuracy: 0.7654
Epoch 416/500
1/1 [==============================] - 0s 48ms/step - loss: 0.0031 - accuracy: 0.7681
Epoch 417/500
1/1 [==============================] - 0s 46ms/step - loss: 0.0031 - accuracy: 0.7676
Epoch 418/500
1/1 [==============================] - 0s 46ms/step - loss: 0.0031 - accuracy: 0.7669
Epoch 419/500
1/1 [==============================] - 0s 46ms/step - loss: 0.0031 - accuracy: 0.7680
Epoch 420/500
1/1 [==============================] - 0s 46ms/step - loss: 0.0031 - accuracy: 0.7667
Epoch 421/500


1/1 [==============================] - 0s 43ms/step - loss: 0.0029 - accuracy: 0.7806
Epoch 494/500
1/1 [==============================] - 0s 44ms/step - loss: 0.0029 - accuracy: 0.7824
Epoch 495/500
1/1 [==============================] - 0s 43ms/step - loss: 0.0029 - accuracy: 0.7816
Epoch 496/500
1/1 [==============================] - 0s 45ms/step - loss: 0.0029 - accuracy: 0.7825
Epoch 497/500
1/1 [==============================] - 0s 44ms/step - loss: 0.0029 - accuracy: 0.7798
Epoch 498/500
1/1 [==============================] - 0s 44ms/step - loss: 0.0029 - accuracy: 0.7826
Epoch 499/500
1/1 [==============================] - 0s 43ms/step - loss: 0.0029 - accuracy: 0.7804
Epoch 500/500
1/1 [==============================] - 0s 43ms/step - loss: 0.0029 - accuracy: 0.7838


In [59]:
testpath = 'data/test'
import cv2
files = os.listdir(testpath)
for idx, file in enumerate(files):
    test = img_to_array(load_img(testpath +str('/')+ file))
    test = resize(test, (224,224), anti_aliasing=True)
    test*= 1.0/255
    lab = rgb2lab(test)
    l = lab[:,:,0]
    L = gray2rgb(l)
    L = L.reshape((1,224,224,3))
    #print(L.shape)
    vggpred = newmodel.predict(L)
    ab = model.predict(vggpred)
    #print(ab.shape)
    ab = ab*128
    cur = np.zeros((224, 224, 3))
    cur[:,:,0] = l
    cur[:,:,1:] = ab
    
    imsave('data/output/'+str(idx)+".jpg", lab2rgb(cur))

C:\Users\Vandit\anaconda3\lib\site-packages\skimage\color\colorconv.py:1109: UserWarning: Color data out of range: Z < 0 in 2398 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))
C:\Users\Vandit\anaconda3\lib\site-packages\skimage\color\colorconv.py:1109: UserWarning: Color data out of range: Z < 0 in 112 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))
C:\Users\Vandit\anaconda3\lib\site-packages\skimage\color\colorconv.py:1109: UserWarning: Color data out of range: Z < 0 in 661 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))
C:\Users\Vandit\anaconda3\lib\site-packages\skimage\color\colorconv.py:1109: UserWarning: Color data out of range: Z < 0 in 7 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))
